<a href="https://colab.research.google.com/github/joynaomi81/English-Yoruba-Machine-Translation/blob/main/English_Yoruba_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers


In [ ]:
!pip install transformers sentencepiece sacremoses
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 54.4 MB/s eta 0:00:00


In [ ]:
# Load your CSV
import pandas as pd
MT_data = "/content/drive/MyDrive/MT.csv"
df = pd.read_csv(MT_data)

# Check first rows
df.head()
df.columns

Index(['English', 'Yoruba'], dtype='object')

In [ ]:
df_sample = df.sample(n=300, random_state=42).reset_index(drop=True)
df_sample.shape

(300, 2)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-1.3B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm
import os

model_name = "facebook/nllb-200-distilled-600M"
src_lang = "eng_Latn"
tgt_lang = "yor_Latn"

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang=src_lang)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

Loading model...


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
def translate_batch(texts, batch_size=8):
    translations = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = [str(t) if pd.notna(t) else "" for t in texts[i:i+batch_size]]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer._convert_token_to_id_with_added_voc(tgt_lang),
            max_length=512
        )
        batch_translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)
    return translations


In [ ]:
def process_file(input_path, output_path1, output_path2):
    print(f"Processing {input_path}...")
    df_sample = pd.read_csv(input_path, sep=",")

In [ ]:
df_translated = df_sample.copy()

print("Translating english column...")

# Translate the english column
df_translated["english_translated"] = translate_batch(df_sample["English"].tolist(), batch_size=8)

# Define the output path
output_path = "/content/drive/MyDrive/translated_MT.csv"

# Save to CSV
df_translated.to_csv(output_path, index=False)

print(f"Saved translated file {output_path}")

Translating english column...


Translating: 100%|██████████| 38/38 [01:14<00:00,  1.95s/it]

Saved translated file /content/drive/MyDrive/translated_MT.csv


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/translated_MT.csv")
df.head(20)

,English,Yoruba,english_translated
0,"Otherwise, they would burn down all the buildi...","Bí bẹ́ẹ̀kọ́, wọn á dáná sun gbogbo ilé inú àgọ...","Tí kò bá rí bẹ́ẹ̀, wọ́n á dáná sun gbogbo ilé ..."
1,"In Turkey, 2.5m students sit university entry ...","Ní orílẹ̀-èdè Turkey, ẹgbẹẹgbẹ̀rún méjì àbọ̀ à...","Ní Tọ́kì, àwọn akẹ́kọ̀ọ́ mílíọ̀nù méjì ààbọ̀ l..."
2,"51. And finally, I will thank all Nigerians ag...","51.Nípàárí, màá tún dúpẹ́ lọ̣́wọ́ gbogbo àwọ̣̣...","51. Níkẹyìn, mo tún máa dúpẹ́ lọ́wọ́ gbogbo àw..."
3,America is so beautiful.,Ìlú Amẹ́ríkà mà rẹwà.,Orílẹ̀-èdè Amẹ́ríkà lọ́nà tó fani mọ́ra.
4,And now she didn't know whether to continue wi...,"Nísinsìnyí, kò mọ̀ bóyá kí ó tẹ̀síwájú pẹ̀lú ọ...",Ó wá sọ pé òun ò mọ bóyá òun máa bá ìjíròrò ná...
5,You can change this in the settings.,O lè yí èyí padà nínú ààtò.,O lè yí èyí padà nínú àwọn àyípadà.
6,The United nations has said that incurable dis...,Àjọ àgbáyé ti ní ààrùn kògbóògùn HIV/AIDS ní í...,Àwọn Orílẹ̀-Èdè Amẹ́ríkà ti sọ pé àrùn àìlera ...
7,"See Labake flopping in an ""under-the-roof cros...",Ẹ wo bí Làbákẹ́ ṣe ń koṣé níbi ìfọ̀rọ̀wánilẹ́n...,"Wo bí Labake ṣe ń ṣubú sínú ""ìwádìí àdììtú láb..."
8,"However, the use of an image from a different ...","Àmọ́ ṣá, àwòrán orílẹ̀-èdè mìíràn tí ó lò láti...","Àmọ́, lílo àwòrán kan láti orílẹ̀-èdè mìíràn m..."
9,---Germany has stated its readiness to partner...,---Orílẹ̀ èdè Germany ti ní ohun ṣetán láti fọ...,---Jámánì ti sọ pé òun múra tán láti bá Nàìjír...


In [ ]:
df.shape

(300, 3)

In [ ]:
!pip install sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
import evaluate

# Load metrics
bleu = evaluate.load("sacrebleu")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

# References and predictions
predictions = df["english_translated"].tolist()
references = [[ref] for ref in df["Yoruba"].tolist()]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# BLEU
print("BLEU:", bleu.compute(predictions=predictions, references=references))

# METEOR
print("METEOR:", meteor.compute(predictions=predictions, references=[r[0] for r in references]))

# BERTScore
print("BERTScore:", bertscore.compute(predictions=predictions, references=[r[0] for r in references], lang="en"))

BLEU: {'score': 8.386999506244681, 'counts': [2894, 973, 415, 200], 'totals': [8747, 8447, 8147, 7847], 'precisions': [33.08562935863725, 11.518882443471055, 5.093899594942924, 2.548744743213967], 'bp': 1.0, 'sys_len': 8747, 'ref_len': 7418}
METEOR: {'meteor': np.float64(0.3068485958530354)}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: {'precision': [0.8987686634063721, 0.8814671039581299, 0.9024465084075928, 0.8377378582954407, 0.8699904084205627, 0.8891423940658569, 0.8818807601928711, 0.8751898407936096, 0.8917491436004639, 0.8880370259284973, 0.8467955589294434, 0.8964316844940186, 0.911686897277832, 0.8908381462097168, 0.8694684505462646, 0.9290804862976074, 0.883576512336731, 0.9136831760406494, 0.8629940152168274, 0.8940004110336304, 0.8791820406913757, 0.8592312335968018, 0.8928970098495483, 0.8909796476364136, 0.8862611651420593, 0.8987144231796265, 0.8973919153213501, 0.8793688416481018, 0.8730636239051819, 0.866066575050354, 0.8989317417144775, 0.8671557307243347, 0.9083108901977539, 0.8801778554916382, 0.8810602426528931, 0.8732898831367493, 0.782821774482727, 0.9182361960411072, 0.8748812675476074, 0.9250091910362244, 0.925284743309021, 0.9011138677597046, 0.8901500701904297, 0.8960345387458801, 0.9612455368041992, 0.8608772158622742, 0.8906855583190918, 0.8365731835365295, 0.8764828443527222,